<a href="https://colab.research.google.com/github/xyz-zy/bert-trc/blob/master/BERT_Pytorch_TREC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODOs

*   Determine the state-of-the art on the dataset (which we are trying to match/beat)
*   Load the model for evaluation from the gcp bucket?
*   Change events/times to one contiguous token instead of wordpieces?
*   Use the data in extra/ to determine if two events are in different sentences
*   Change script to get TimeBank-Dense examples
*   Implement training from latest checkpoint????



In [0]:
#!pip3 install torchvision

In [0]:
!pip3 install -q pytorch-pretrained-bert

In [0]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import zipfile
# from matplotlib import pyplot as plt
# %matplotlib inline
import sys
import datetime
import json
import torch
import sys

In [0]:
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
#We will use the most basic of all of them
repo = 'model_repo'
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{repo}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{repo}/outputs'

MODEL_NAME = 'bert-base-uncased'
print(f'***** Model output directory: {OUTPUT_DIR} *****')
print(f'***** BERT pretrained directory: {BERT_PRETRAINED_DIR} *****')

***** Model output directory: model_repo/outputs *****
***** BERT pretrained directory: model_repo/uncased_L-12_H-768_A-12 *****


In [0]:
# assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
# TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# print('TPU address is', TPU_ADDRESS)

# from google.colab import auth
# auth.authenticate_user()

# with tf.Session(TPU_ADDRESS) as session:
#   print('TPU devices:')
#   pprint.pprint(session.list_devices())

#   # Upload credentials to TPU.
#   with open('/content/adc.json', 'r') as f:
#     auth_info = json.load(f)
#   tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
#   # Now credentials are set for all future sessions on this TPU.

In [0]:
BUCKET = 'bert-tre-2' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
BUCKET_OUTPUT_DIR = 'gs://{}/bert_squad/outputs'.format(BUCKET)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: model_repo/outputs *****


In [0]:
USERNAME = 'bert-trex' #@param {type:"string"}
PASSWORD = '9udYXiKkqGhWJ39' #@param {type:"string"}
os.environ['GITHUB_AUTH'] = USERNAME + ':' + PASSWORD

!rm -r ./bert-temporal-relation-extraction
!rm -r ./timeml
!git clone https://$GITHUB_AUTH@github.com/xyz-zy/bert-temporal-relation-extraction 
!mv bert-temporal-relation-extraction/* .

Cloning into 'bert-temporal-relation-extraction'...
remote: Enumerating objects: 433, done.
remote: Counting objects: 100% (433/433), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 433 (delta 389), reused 382 (delta 351), pack-reused 0
Receiving objects: 100% (433/433), 1.28 MiB | 3.73 MiB/s, done.
Resolving deltas: 100% (389/389), done.
mv: cannot move 'bert-temporal-relation-extraction/extra' to './extra': Directory not empty


In [0]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

import pytorch_pretrained_bert
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.modeling import BertPreTrainedModel
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert import file_utils
# import run_squad
import timeml
from timeml import ExampleLoader
import collections

In [0]:
loader = ExampleLoader()
# timeml_examples = loader.read_examples_from_directory("timeml/")
train_examples, dev_examples = loader.read_example_files("timeml/")

e41 timeml/APW19980213.1380.tml
e40 timeml/APW19980213.1380.tml
e39 timeml/APW19980213.1380.tml
['AFTER', 'BEFORE', 'BEGINS', 'BEGUN_BY', 'DURING', 'DURING_INV', 'ENDED_BY', 'ENDS', 'IAFTER', 'IBEFORE', 'IDENTITY', 'INCLUDES', 'IS_INCLUDED', 'SIMULTANEOUS']
14


In [0]:
loss_weights = {}
for label in loader.label_list:
  loss_weights[label] = 0

for ex in train_examples:
  loss_weights[ex.str_label] += 1

num_examples = len(train_examples)
for key in loss_weights:
  loss_weights[key] = num_examples / loss_weights[key]
  
weights_list = [float("%3.f" % loss_weights[key]) for key in loader.label_list]
print(weights_list) # The "DURING_INV" label only has one occurrence

[7.0, 5.0, 106.0, 91.0, 22.0, 6246.0, 37.0, 87.0, 160.0, 184.0, 9.0, 11.0, 5.0, 10.0]


In [0]:
# train_examples = timeml_examples[:-100]
# len(train_examples)

In [0]:
print(len(train_examples))
loader.antithetics(train_examples)
print(len(train_examples))

8770

In [0]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True)

In [0]:
# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 16
PREDICT_BATCH_SIZE = 8
LEARNING_RATE =2e-5
NUM_TRAIN_EPOCHS = 4.0
WARMUP_PROPORTION = 0.1
GRADIENT_ACCUMULATION_STEPS = 1
MAX_SEQ_LENGTH = 200#5000
DOC_STRIDE=128
LOCAL_RANK = -1
N_BEST_SIZE = 20
MAX_ANSWER_LENGTH = 30
VERSION_2_WITH_NEGATIVE = False
NULL_SCORE_DIFF_THRESHOLD = 0.0
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

In [0]:
num_train_optimization_steps = int(len(train_examples) / TRAIN_BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS) * NUM_TRAIN_EPOCHS

In [0]:
class BertForTRE(BertPreTrainedModel):
    
    def __init__(self, config):
        super(BertForTRE, self).__init__(config)
        self.bert = BertModel(config)
        self.apply(self.init_bert_weights)
        self.classifier = torch.nn.Linear(2*768, 14)
        self.train_class_loss_weights=np.array(weights_list)
        self.loss = torch.nn.CrossEntropyLoss(
            weight=torch.from_numpy(
              self.train_class_loss_weights)
                  .float()
        )
        
    def get_positions(self, sequence_output, positions):
        output_tensors = []
        for sample_idx, pos in enumerate(positions):
            position_tensor = sequence_output[sample_idx][pos]
            output_tensors.append(position_tensor)
        return torch.stack(output_tensors, dim=0)
            
    
    def gen_mask(positions):
        mask = torch.zeros(len(positions), len(positions), dtype=torch.uint8)
        for sample_idx, pos in enumerate(positions):
            mask[sample_idx, pos] = 1
        return mask
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, tre_labels=None, e1_pos=None, e2_pos=None):
        sequence_output, _ = self.bert(input_ids, token_type_ids, attention_mask,
                                       output_all_encoded_layers=False)

        
        ##[batch_size,sequence_length=1,hidden_size]
        e1_hidden = self.get_positions(sequence_output, e1_pos)
        e2_hidden = self.get_positions(sequence_output, e2_pos)
        
        
        cls_tensor = torch.cat((e1_hidden,e2_hidden),1)
  
#         print(input_ids.shape)
#         print(e1_hidden.shape)
#         print(e2_hidden.shape)
#         print(cls_tensor.shape)

      
        out = self.classifier(cls_tensor)
        loss = self.loss(out, tre_labels)
        return out, loss
       

In [0]:
model = BertForTRE.from_pretrained(MODEL_NAME,
                cache_dir=os.path.join(str(file_utils.PYTORCH_PRETRAINED_BERT_CACHE), 'distributed_{}'.format(-1)))

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device, n_gpu)

cuda 1


In [0]:
model.to(device)

BertForTRE(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediat

In [0]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())

# hack to remove pooler, which is not used
# thus it produce None grad that break apex
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]

no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

In [0]:
optimizer = BertAdam(optimizer_grouped_parameters,
                   lr=LEARNING_RATE,
                   warmup=WARMUP_PROPORTION,
                   t_total=num_train_optimization_steps)

In [0]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 #doc_span_index,
                 tokens,
                 #token_to_orig_map,
                 #token_is_max_context,
                 input_ids,
                 input_mask,
                 segment_ids,
                 label,
                 e1_position=None,
                 e2_position=None,
                 #,is_impossible=None
                  ):
        self.unique_id = unique_id
        self.example_index = example_index
        self.tokens = tokens
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label = label
        self.e1_position = e1_position
        self.e2_position = e2_position

        
def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 doc_stride, is_training):
    """Loads a data file into a list of `InputBatch`s."""

    unique_id = 1000000000

    features = []
    
                
    ##Generates features from examples
    #print(len(examples))
    for (example_index, example) in enumerate(examples):
        input_tokens = tokenizer.tokenize(example.text)
        
        max_tokens_for_doc = max_seq_length - 3

        if len(input_tokens) > max_tokens_for_doc:
            unique_id += 1
            continue
            #input_tokens = input_tokens[0:max_tokens_for_doc]

        tok_to_orig_index = []
        orig_to_tok_index = []
        all_doc_tokens = []
        for (i, token) in enumerate(example.text.split()):
            orig_to_tok_index.append(len(all_doc_tokens))
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                tok_to_orig_index.append(i)
                all_doc_tokens.append(sub_token)

        # + 1 accounts for CLS token
        tok_e1_pos = orig_to_tok_index[example.e1_pos] + 1
        tok_e2_pos = orig_to_tok_index[example.e2_pos] + 1
       

        # The -3 accounts for [CLS], [SEP] and [SEP]

        tokens = []
        #token_to_orig_map = {}
        token_is_max_context = {}
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in input_tokens:
            tokens.append(token)
            segment_ids.append(0)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert tok_e1_pos < max_seq_length
        assert tok_e2_pos < max_seq_length
 
        features.append(
            InputFeatures(
                unique_id=unique_id,
                example_index=example_index,
                tokens=tokens,
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                label=example.int_label,
                e1_position=tok_e1_pos,
                e2_position=tok_e2_pos
            )
        )
        unique_id += 1

    return features

In [0]:
train_features = convert_examples_to_features(
            examples=train_examples,
            tokenizer=tokenizer,
            max_seq_length=MAX_SEQ_LENGTH,
            doc_stride=DOC_STRIDE,
            is_training=True)
print(len(train_features))

7351


In [0]:
feature = train_features[4]
print(feature.tokens)
print(len(feature.tokens))
print(len(feature.input_ids))
print(feature.e1_position)
print(feature.e2_position)
print(feature.tokens[feature.e1_position])
print(feature.tokens[feature.e2_position])
print(len(train_features))

['[CLS]', '10', '/', '26', '/', '89', 'dominion', 'textile', 'inc', '.', 'holders', 'adopted', 'a', 'shareholder', '-', 'rights', 'plan', 'at', 'the', 'annual', 'meeting', '.']
22
200
20
1
meeting
10
7351


In [0]:
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

all_label_ids = torch.tensor([e.label for e in train_features], dtype=torch.long)
all_e1_pos = torch.tensor([e.e1_position for e in train_features], dtype=torch.long)
all_e2_pos = torch.tensor([e.e2_position for e in train_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_e1_pos, all_e2_pos)

train_sampler = RandomSampler(train_data)

In [0]:
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [0]:
model.train()

BertForTRE(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediat

In [0]:
!set CUDA_LAUNCH_BLOCKING=1

In [0]:
from tqdm import tqdm, trange
global_step = 0
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration", disable=LOCAL_RANK not in [-1, 0])):
        if n_gpu == 1:      
          batch = tuple(t.to(device) for t in batch) # multi-gpu does scattering it-self
        #input_ids, token_type_ids, attention_mask, tre_labels, e1_pos, e2_pos = batch
        #print(*batch)
        _, loss = model(*batch)#model(input_ids, token_type_ids, attention_mask, tre_labels, e1_pos, e2_pos)

#         if n_gpu > 1:
#             #loss = loss.mean() # mean() to average on multi-gpu.
        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

#         if args.fp16:
#             optimizer.backward(loss)
#         else:
        loss.backward()
        print("%s" % loss)
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
#             if args.fp16:
#                 # modify learning rate with special warm up BERT uses
#                 # if args.fp16 is False, BertAdam is used and handles this automatically
#                 lr_this_step = args.learning_rate * warmup_linear(global_step/num_train_optimization_steps, args.warmup_proportion)
#                 for param_group in optimizer.param_groups:
#                     param_group['lr'] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

Iteration:   0%|          | 1/230 [00:01<05:06,  1.34s/it]

tensor(2.8102, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   1%|          | 2/230 [00:02<04:59,  1.31s/it]

tensor(2.8128, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration:   1%|▏         | 3/230 [00:03<04:54,  1.30s/it]

...

tensor(1.2745, device='cuda:0', grad_fn=<NllLossBackward>)



Iteration: 100%|█████████▉| 229/230 [05:21<00:01,  1.41s/it]

tensor(0.6999, device='cuda:0', grad_fn=<NllLossBackward>)



Epoch: 100%|██████████| 4/4 [21:25<00:00, 320.75s/it]

tensor(0.8676, device='cuda:0', grad_fn=<NllLossBackward>)


In [0]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model itself

In [0]:
def to_json_file(config, json_file_path):
        """ Save this instance to a json file."""
        with open(json_file_path, "w", encoding='utf-8') as writer:
            writer.write(config.to_json_string())

In [0]:
def save_vocabulary(tokenizer, vocab_path):
    """Save the tokenizer vocabulary to a directory or file."""
    index = 0
    if os.path.isdir(vocab_path):
        vocab_file = os.path.join(vocab_path, 'vocab.txt')
    with open(vocab_file, "w", encoding="utf-8") as writer:
        for token, token_index in sorted(tokenizer.vocab.items(), key=lambda kv: kv[1]):
            if index != token_index:
                logger.warning("Saving vocabulary to {}: vocabulary indices are not consecutive."
                               " Please check that the vocabulary is not corrupted!".format(vocab_file))
                index = token_index
            writer.write(token + u'\n')
            index += 1
    return vocab_file

In [0]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [0]:
# If we save using the predefined names, we can load using `from_pretrained`

output_model_file = os.path.join(OUTPUT_DIR, "pytorch_model.bin")
output_config_file = os.path.join(OUTPUT_DIR, "bert_config.json")

torch.save(model_to_save.state_dict(), output_model_file)
to_json_file(model_to_save.config, output_config_file)
#model_to_save.config.to_json_file(output_config_file)
save_vocabulary(tokenizer, OUTPUT_DIR)#tokenizer.save_vocabulary(OUTPUT_DIR)

'model_repo/outputs/vocab.txt'

In [0]:
# Please don't run this command; the current model in the bucket gets 0.67 F1 on dev set after 4 epochs of training
#!gsutil cp ./model_repo/outputs/* gs://bert-tre/bert_squad/outputs/

Copying file://./model_repo/outputs/bert_config.json [Content-Type=application/json]...
Copying file://./model_repo/outputs/nbest_predictions.json [Content-Type=application/json]...
Copying file://./model_repo/outputs/predictions.json [Content-Type=application/json]...
Copying file://./model_repo/outputs/pytorch_model.bin [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil

#Evaluate Model

In [0]:
# !gsutil cp gs://bert-tre/bert_squad/outputs/* ./model_repo/outputs/

In [0]:
# Load a trained model and vocabulary that you have fine-tuned
model = BertForTRE.from_pretrained(OUTPUT_DIR)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR, do_lower_case=DO_LOWER_CASE)

In [0]:
model.to(device)

BertForTRE(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIntermediat

In [0]:
#dev_examples = loader.read_examples_from_directory("timeml/")
# dev_examples = timeml_examples[-100:]
loader.antithetics(dev_examples)
print(len(dev_examples))

dev_features = convert_examples_to_features(
    examples=dev_examples,
    tokenizer=tokenizer,
    max_seq_length=MAX_SEQ_LENGTH,
    doc_stride=DOC_STRIDE,
    is_training=False)

245


In [0]:
all_input_ids = torch.tensor([f.input_ids for f in dev_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in dev_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in dev_features], dtype=torch.long)

all_label_ids = torch.tensor([e.label for e in dev_features], dtype=torch.long)
all_e1_pos = torch.tensor([e.e1_position for e in dev_features], dtype=torch.long)
all_e2_pos = torch.tensor([e.e2_position for e in dev_features], dtype=torch.long)

dev_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids, all_e1_pos, all_e2_pos)

# Run prediction for full data
dev_sampler = SequentialSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=PREDICT_BATCH_SIZE)

In [0]:
model.eval()
all_results = []
all_labels = []
print("Start evaluating")
correct = 0
total = 0
for input_ids, input_mask, segment_ids, label_id, e1_pos, e2_pos in tqdm(dev_dataloader, desc="Evaluating", disable=LOCAL_RANK not in [-1, 0]):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_id = label_id.to(device)
    e1_pos = e1_pos.to(device)
    e2_pos = e2_pos.to(device)
    with torch.no_grad():
        out, _ = model(input_ids, input_mask, segment_ids, label_id, e1_pos, e2_pos)
        val, guess_idxs = out.max(1)
        
        print(guess_idxs)
       
        for idx, guess in enumerate(guess_idxs):
            all_results.append(guess.item())
            all_labels.append(label_id[idx].item())
            if guess == label_id[idx]:
              correct += 1
        total += len(out)
        print(correct, total)


Evaluating:   4%|▎         | 1/28 [00:00<00:04,  6.14it/s]

Start evaluating
tensor([ 0,  0, 13, 12, 12,  1,  7,  6], device='cuda:0')
4 8


Evaluating:  11%|█         | 3/28 [00:00<00:03,  6.91it/s]

tensor([12,  6,  0, 13, 12, 12, 12, 12], device='cuda:0')
10 16
tensor([ 1, 12, 12, 10, 12,  1,  1,  1], device='cuda:0')
15 24


Evaluating:  18%|█▊        | 5/28 [00:00<00:02,  7.75it/s]

...

tensor([10,  1, 13, 11,  0,  0, 13,  0], device='cuda:0')
109 176
tensor([0, 1, 0, 1, 0, 0, 0, 0], device='cuda:0')
113 184


Evaluating:  89%|████████▉ | 25/28 [00:02<00:00,  8.61it/s]

tensor([4, 1, 0, 1, 0, 0, 1, 0], device='cuda:0')
120 192
tensor([1, 0, 1, 0, 1, 0, 1, 1], device='cuda:0')
128 200


Evaluating: 100%|██████████| 28/28 [00:03<00:00,  8.63it/s]

tensor([0, 0, 1, 0, 4, 1, 0, 0], device='cuda:0')
136 208
tensor([ 0, 13,  1,  4,  4,  0,  0,  4], device='cuda:0')
142 216
tensor([4, 4], device='cuda:0')
144 218


In [0]:
print("accuracy: ", correct/total)

print(len(dev_features), len(train_features))

accuracy:  0.6605504587155964
218 7351


In [0]:
print(all_labels)
print(all_results)

[12, 0, 1, 12, 12, 1, 13, 13, 12, 6, 1, 1, 12, 12, 12, 12, 1, 12, 12, 12, 13, 1, 2, 1, 1, 13, 13, 6, 13, 10, 13, 0, 12, 12, 6, 13, 0, 0, 12, 0, 12, 0, 7, 7, 13, 11, 10, 1, 12, 12, 1, 4, 1, 12, 0, 0, 12, 11, 13, 0, 1, 13, 12, 12, 12, 12, 12, 12, 12, 0, 6, 12, 12, 1, 1, 0, 1, 4, 13, 4, 13, 1, 0, 1, 4, 4, 12, 12, 1, 4, 0, 0, 0, 1, 1, 0, 1, 0, 12, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 4, 0, 13, 13, 13, 7, 12, 1, 12, 12, 13, 6, 12, 7, 10, 1, 1, 6, 13, 13, 3, 1, 10, 1, 4, 4, 13, 1, 1, 4, 10, 13, 4, 13, 13, 4, 10, 13, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 4, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 4, 4, 0, 1, 0, 4, 4, 0, 4, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 4, 1, 0, 0, 0, 0, 0, 4, 4, 0, 0, 4, 4, 4]
[0, 0, 13, 12, 12, 1, 7, 6, 12, 6, 0, 13, 12, 12, 12, 12, 1, 12, 12, 10, 12, 1, 1, 1, 13, 13, 13, 6, 6, 13, 0, 0, 12, 1, 6, 1, 0, 13, 12, 12, 12, 0, 4, 4, 0, 10, 1, 1, 12, 12, 0, 4, 13, 4, 0, 13, 0, 13, 13, 0, 13, 13, 12, 12, 12, 12, 12, 12, 13, 12, 13, 10, 12, 1, 1, 0, 1, 11, 

In [0]:
import sklearn.metrics
ls = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
sklearn.metrics.f1_score(all_labels, all_results, labels=ls, average='weighted')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6775847784749126

In [0]:
print("| %.3f | %.3f | %.3f | %.3f |" % (train_accuracy, precision, recall, f1))